In [57]:
%%writefile train_iris.py

import argparse
import json
import random
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

def main():
    # Parse arguments
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_estimators', type=int, default=10)
    parser.add_argument('--max_depth', type=int, default=None)
    args = parser.parse_args()

    # Load dataset
    data = load_iris()
    X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

    # Train model
    model = RandomForestClassifier(n_estimators=args.n_estimators, max_depth=args.max_depth)
    model.fit(X_train, y_train)

    # Evaluate model
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')

    # Log the F1 score for SageMaker HPO
    print(f"#quality_metric: host=unknown, train f1_score <score>={f1}")
    
    # Log the F1 score for SageMaker HPO - to mimic validation for objective metric
    # Generate random f1_score between 0 and 1 (you can adjust the range as needed)
    valid_f1 = random.uniform(0, 1)
    print(f"#quality_metric: host=unknown, valid f1_score <score>={valid_f1}")

if __name__ == "__main__":
    main()


Overwriting train_iris.py


In [37]:
## usual ONE training job without any hyperparameter tuning job as parent

import sagemaker
from sagemaker.sklearn import SKLearn
from sagemaker import get_execution_role

# Get SageMaker execution role
role = get_execution_role()

output_path = "s3://apartment-pricing/model/random_forest/"

sklearn_estimator = SKLearn(
    entry_point="train_iris.py",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    hyperparameters={
        "max_depth": 10,
        "n_estimators": 10,
    },
    output_path=output_path,
    framework_version="0.23-1",
)


In [38]:
# Launch training job
sklearn_estimator.fit()

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-12-04-22-04-47-650


2024-12-04 22:04:49 Starting - Starting the training job...
2024-12-04 22:05:04 Starting - Preparing the instances for training...
2024-12-04 22:05:36 Downloading - Downloading input data...
2024-12-04 22:05:56 Downloading - Downloading the training image...
2024-12-04 22:06:47 Training - Training image download completed. Training in progress.
2024-12-04 22:06:47 Uploading - Uploading generated training model2024-12-04 22:06:38,299 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-12-04 22:06:38,303 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-04 22:06:38,351 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-12-04 22:06:38,510 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-04 22:06:38,523 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-04 22:06:38,536 sagemaker-training-toolkit INFO

In [52]:
# hyperparameter tuning job --> calls upto 10 training jobs
# metric_definitions VIMP ... has to follow the format as printed in train.py. The job regex it from the cloudwatch log

import sagemaker
from sagemaker.sklearn import SKLearn
from sagemaker.tuner import HyperparameterTuner, IntegerParameter
from sagemaker import get_execution_role

# Get SageMaker execution role
role = get_execution_role()

# S3 path for output
output_path = "s3://apartment-pricing/model/random_forest/"

# Define the SKLearn estimator
sklearn_estimator = SKLearn(
    entry_point="train_iris.py",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    framework_version="1.2-1",  # Updated to the latest version
    output_path=output_path,
)

# Define hyperparameter ranges
hyperparameter_ranges = {
    "n_estimators": IntegerParameter(50, 200),
    "max_depth": IntegerParameter(5, 20),
}

# Define objective metric
objective_metric_name = "valid_f1_score"
objective_type = "Maximize"

# Create HyperparameterTuner object
tuner = HyperparameterTuner(
    estimator=sklearn_estimator,
    objective_metric_name="valid_f1_score",
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=10,  # Total number of training jobs
    max_parallel_jobs=2,  # Parallel training jobs
    objective_type=objective_type,
    metric_definitions=[
    {"Name": "valid_f1_score", 
     "Regex": "#quality_metric: host=unknown, valid f1_score <score>=(.*)"},
    ],
)

# Start hyperparameter tuning
tuner.fit()


INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-scikit-lea-241204-2309


..........................................................................!


In [56]:
# gets details of the tuning job

import boto3

client = boto3.client('sagemaker')

response = client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName='sagemaker-scikit-lea-241204-2309'
)

print(response)


{'HyperParameterTuningJobName': 'sagemaker-scikit-lea-241204-2309', 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:442426877041:hyper-parameter-tuning-job/sagemaker-scikit-lea-241204-2309', 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian', 'HyperParameterTuningJobObjective': {'Type': 'Maximize', 'MetricName': 'valid_f1_score'}, 'ResourceLimits': {'MaxNumberOfTrainingJobs': 10, 'MaxParallelTrainingJobs': 2}, 'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'n_estimators', 'MinValue': '50', 'MaxValue': '200', 'ScalingType': 'Auto'}, {'Name': 'max_depth', 'MinValue': '5', 'MaxValue': '20', 'ScalingType': 'Auto'}], 'ContinuousParameterRanges': [], 'CategoricalParameterRanges': []}, 'TrainingJobEarlyStoppingType': 'Off'}, 'TrainingJobDefinition': {'StaticHyperParameters': {'_tuning_objective_metric': 'valid_f1_score', 'sagemaker_container_log_level': '20', 'sagemaker_estimator_class_name': '"SKLearn"', 'sagemaker_estimator_module': '"sagemaker.sklearn.estimator"',

In [55]:
print(response['BestTrainingJob'])

{'TrainingJobName': 'sagemaker-scikit-lea-241204-2309-007-364b6d61', 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:442426877041:training-job/sagemaker-scikit-lea-241204-2309-007-364b6d61', 'CreationTime': datetime.datetime(2024, 12, 4, 23, 12, 58, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2024, 12, 4, 23, 13, 3, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2024, 12, 4, 23, 13, 32, tzinfo=tzlocal()), 'TrainingJobStatus': 'Completed', 'TunedHyperParameters': {'max_depth': '13', 'n_estimators': '50'}, 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'valid_f1_score', 'Value': 0.8101555705070496}, 'ObjectiveStatus': 'Succeeded'}
